# Introduction
This IPython notebook contains some general information about the project as well as a rough outline of how problems were tackled.

## Structure of this documentation

## Situation

_[ReadyLlingua](https://readylingua.com/)_ is a Switzerland-based company selling products for language learning. It does so by providing tools to match up a audio or video file with a given transcript, which has been created manually. The transcript is augmented with temporal information so that the viewer/listener can easily follow what's being said by following the highlighted parts of the transcript. This process is called Forced Alignment and is also used for example in subtitles or Karaoke.

Currently, _ReadyLingua_ uses a proprietary product called _Speech Indexer_ to align text with audio. However, this process is cumbersome and requires a lot of manual effort (one hour per 15 minutes aligned audio according to ReadyLingua). _ReadyLlingua_ also built a prototype which has been evaluated on evaluating audio/transcripts in Swiss German. However, this prototype only yielded an accuracy of 66-80% of correctly idintified phonemes.

## Project goal and scope

Within the context of a research project supported by the Swiss Innovation Agency ([InnoSuisse](https://www.innosuisse.ch)) methods shall be evaluated as of how the alignment process could be improved or even completely automated. The project application (application number 29417.1) foresees three different approaches:

* **Recognition of speech segments by identifying speech pauses and phonemes:** This approach uses a phonetic transcript, in which the individual sounds (phonemes) of an utterance are mapped to characters of the [International Phonetic Alphabet](http://internationalphoneticalphabet.org) (IPA). Speech pauses are used to segment the transcript by analyzing the phonemes shortly before and after a speech pause (context). The transcript is also mapped to IPA characters. Alignment is done by comparing the two IPA-transcripts. This approach is highly dependent on accurate transcripts.
* **Recognition of speech segments by applying a Speech-to-Text (STT) application:** In this approach, an existing speech recognition solution (such as Google Speech) is used to automatically generate a transcript for a given audio file. The temporal information is usually included in such transcripts. Since this process does not use the manual transcript, it is highly dependent on the performance of the STT application. Since the manual transcript does not always embrace all utterances (e.g. repetitions, slips of the tongue, interjections), the generated text may or may not match up with the manual transcript, even if the STT application produces a perfect transcript for the audio file.
* **Recognition of speech segments by means of a Recurrent Neural Network (RNN):** In this approach the alignment is done by applying methods from Deep Learning. It uses a RNN to learn the segmentation from spectrograms from the spectrograms of the audio clips.

The InnoSuisse project is below referred to as _main project_. This project focuses on the last approach. It does not consider the other two and is referred to simply as _project_

### Motivation

Automatically detecting speech segments should reduce the effort needed to manually align text with speech. By using machine learning should also reduce the sensitivity to inaccurate transcripts.

## Plan and Hypothesis

The original idea was to detect speech pauses in an audio file and then map those to positions in a transcript in order to get a segmentation. However, since speech pauses do by definition not contain any relevant information for alignment (since there is no corresponding text), the plan had to be changed slightly. The changed approach still aims for detecting speech pauses, but instead of directly exploiting those, a transcript for the speech segments (i.e. the parts of the audio signal between the speech pauses) shall be generated. These transcripts shall then be compared with the manual transcript (which comprises the transcripts of all speech segments).

The changed approach is more similar to the second approach as it uses some sort of TTS engine to generate a transcript. However, it is less dependent on the performance of said engine because it also uses the information from the manual transcript, whose quality is deemed perfect. The hypothesis for this project is that it should be easier to align whole segments of a text with a gold standard, rather than individual characters or phonemes. Thus this approach is supposed to yield good results even for partially imperfect text segments. 

### Processing pipeline

The designated approach contains two main tasks: Automatic Speech Recognition (ASR) and Local Sequence Alignment (LSA). It also needs some way to detect when the audio signal contains speech (Voice Activity Detection, VAD) as well as some preprocessing.

Since these tasks operate highly independent of each other, I refrained from implementing an End-to-end (E2E) approach which comprises all steps necessary in order to align text and audio within a single model. Instead I went for a pipeline that chains all those steps. The stages of such a pipeline can be individually optimized or replaced. Dividing the process into a sequence of stages also opens up to methods like _ceiling analysis_ to find out which stages need most improvement. 

The following figure shows the stages identified.

* **Stage 1 - Preprocessing:** This stage processes the raw input data from different sources and brings it into a standardized format that is suitable for processing in subsequent stages. The stage encompasses tasks for audio resampling, text normalization and error correction. 
* **Stage 2 - Voice Activity Detection (VAD):** This stage processes the audio signal from pairs of audio and transcript. Its purpose is to extract the speech parts from an signal. The signal is therefore divided into individual speech segments which can then be fed to a pre-trained RNN.
* **Stage 3 - Automatic Speech Recognition (ASR):** This stage processes speech segments and generates their transcripts. As defined by the approach, a RNN is used to do this.
* **Stage 4 - Local Sequence Alignment (LSA):** The last step of the pipeline compares the generated transcripts with the manually created transcript and tries to align them. It does so by comparing each (possibly imperfect) speech segment transcript and trying to determine its position in the original transcript by means of a suitable measure.

Despite being less dependent on the performance of a STT engine, ASR is still at the heart of the pipeline. Hence it is expected that the performance of the RNN is crucial for the overall performance. 

### Challenges and Risks
Pipelining the process reduces the overall risk of failure by restraining it to the performance of the individual stages. However, there are still a lot of challenges to overcome. Most of them are rooted in the difficulty of speech recognition. There are tools and frameworks available for ASR (such as [Kaldi](http://kaldi-asr.org/)) and Forced Alignment (such as [Gentle](https://lowerquality.com/gentle/)), but those still require to train an ASR-model. Pre-trained ASR-models (e.g. [Kaldi+PDNN](https://www.cs.cmu.edu/~ymiao/kaldipdnn.html)) or example code from papers are sometimes available. Most high-performance models (like [Google Speech](https://cloud.google.com/speech-to-text/)) however, are company proprietary and though sometimes made publicly accessible via API, they can be considered a black box and not tunable. Plugging in such a system would be possible with the pipelining approach, but that would then correspond more with the second approach of the main project.

The challenges of training an ASR system can be subdivided into two categories:

* **acoustic challenges**: Those challenges come from properties of the audio signal or the speaker.
* **linguistic challenges**: Challenges in this category arise from properties of language itself. 
* **Conceptual**: Challenges that derive from the use of an RNN for ASR

Examples for all three categories are explained in more detail in the following paragraphs.

#### Acoustic challenges
Properties of the audio signal have a great influence on the learning process of a RNN. Audio data with lot of background noise makes it much harder for an ASR system to recognize speech. Other examples for properties of the audio signal are quality (sampling rate) or the presence of distortion (echo, reverb).

Another source of errors are properties of the speaker. Consider for example the speaking rate: The features for an audio signal from slow speaker are completely different to the audio signal of a fast speaker, despite the underlying text being identical. Another example is speaker gender: audio recordings of female readers usually have a much higher pitch than recordings from male readers, however the text is still the same.

#### Linguistic challenges
Written language often contains redundancy, i.e. information that is present multiple times (redundancy by duplication) or characters that do not actually carry any relevant information for ASR (no entropy). Consider for example the fact that the symbols `A` and `a` are two representations of the same character (redundancy by duplication) or the fact that punctuation symbols like `.`, `!`, `?` etc. are not pronounced explicitly (no entropy). Redundancy is also present in the fact that there may be more than one spelling variant for the same sound (such as `t` and `tt`, `z` and `tz`, `f` and `ph` or `k` and `ck`) or some sounds are represented by more than one character (e.g. `sch`, `ch` or `ng`).

On the other hand, there is a lot of ambiguity in spoken language. Because different languages have different pronunciation patterns, there is no one-to-one mapping from a certain character to a certain phoneme. This is especially important in languages like French or English, where the pronunciation of a word can be quite different from its written representation. Thus ASR is highly language-dependent. Also there may be two words with the same pronunciation, but different spelling (called **[homophones](https://en.wikipedia.org/wiki/Homophone)**, such as the ever-famous _their_ and _there_) or words even that have the same spelling but different pronunciation (called **[homographs](https://en.wikipedia.org/wiki/Homograph)**, like _lead_, which is pronounced differently when referring to the metal than when referring to the act of leading). To make things even harder, there are even words that are the same both in spelling and pronunciation (called **[homonyms](https://en.wikipedia.org/wiki/Homonym)**, such as the word _bear_ which refers to an animal when used as a noun and the act of carrying when used as a verb).

#### Conceptual challenges
Sequence labelling describes the process of assigning a label to a sequence of tokens. In this case, the tokens are parts of the audio signal and the labels are the characters of the alphabet. The characters must then be post-processed somehow to transform the sequence labels into a valid transcript.

RNNs are particularly usefuls variants of Neural Networks to deal with sequential data. They operate of labelled sequences whereas the sequence tokens are referred to as the input $X = [x_1, ..., x_T]$ and the sequence labels as the output $Y = [y_1, ..., y_U]$. In this case, both input and output are sequences (i.e. $T>1, U>1$) which is also called a _many-to-many_ architecture (a.k.a. _sequence-to-sequence_ model). The input tokens are the frames of a spectrogram and the labels the character of a transcript. The learning process is supervised, i.e. the actual transcript is used to learn the mapping from a frame to a character. This implies that the input length must not be longer than the output length ($T>U$). This precondition is not a problem in this case, since the length of the input sequence (i.e. the number of  frames in a spectrogram) is usually in the hundreds or thousands whereas the length of the output sequence (i.e. the number of characters in the corresponding transcript) is usually only a few dozens.

However, since the speech segments are not equally sized, the input sequences $X$ have different lengths between different samples. Since labels correspond to arbitrary transcripts, output sequences $Y$ have varying lengths too. Not even the ratio between input and output length remains constant between samples (with the constraint $T>U$ explained above).

Also, the framework must deal with the fact that both the length of the inputs and outputs usually vary between samples since different speaking rates result in different sequence lengths.

Finally, it is not possible to map directly from frame to character, because the exact alignment is unknown (i.e. we do not know where in the signal a particular character starts or ends). The only thing that's known is the order of the characters.

## Related Research
ASR is a very wide and active field of research. Traditional approaches involved mapping parts of the audio signal to phonemes. However, such approaches required a phonetic transcript which involved a human expert who defined one or more variants of how a particular word was pronunciated. The features derived from the audio signal were then compared to this pronunciation table by calculating the sequence of words that would match up with the features with the highest probability. The use of pronunciation tables made the system highly sensitive to things like accent or words that were not contained in the table. Also, such a system often constisted of several components like a language model (who would model the probabilities of sequences of characters and/or words via n-grams), a pronunciation model (said pronunciation table) and an acoustic model (who would model the relationship of audio features to phonemes). Bringing all those components to nicely work together often required a fair amount of fine-tuning which was not easy because the components had to be trained with different objectives and were still highly interdependent (errors in one component would affect other components). These issues together with the strong dependence on expert knowledge were supposedly responsible for the poor performance of such systems.

With the rise of Deep Learning the trend shifted towards end-to-end models that did not require a lot of manual fine tuning. Such models would learn the relationship between audio features and transcript without the need of intermediate components or an additional representation layer (like a phonetic transcription). One particular concept that had huge impact was _Connectionist Temporal Classification_ (CTC) which does not require a prior alignment of audio and text <cite data-cite="6174726/STDCGILD"></cite>. Training an RNN with CTC also overcomes a lot of the technical challenges formulated above because it does not require  a previous alignment of frames and characters for training in order to learn how to transcribe an unknown audio signal later.

A lot of recent research focuses on training neural networks with CTC. Because of the importance of CTC and also because CTC was used in the ASR-stage of the pipeline presented above, it is explained in a bit more detail below.

### Connectionist Temporal Classification (CTC)
The following explanations give an quick overview of how CTC works. There is a very illustrative explanation of CTC on [Distill](https://distill.pub/2017/ctc/) <cite data-cite="6174726/IAIAA92K"></cite>, from which this overview is derived. If an in-depth explanation is needed, it may be worth reading [the original paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.75.6306&rep=rep1&type=pdf).

#### Alignments and output sequence

CTC is _alignment-free_, i.e. it does not require the input and output to be aligned. Instead, it assigns each input token (=frame in this case) an output label (=character in this case). This results in an _alignment_, which is a sequence of labels with the same length as the input sequence. The possible labels can be anything, for speech recognition they are usually characters from an alphabet. In the context of this project the labels are lowercase characters of the latin alphabet, without any special characters. Additionally, CTC will require an additional label (called the _blank label_ $\epsilon$) to label unknown tokens or silend parts. This means in this project CTC is used with a label set of 27 characters (`a-z` and the blank token). Consider the following alignments for a sequence of six tokens which was derifed for a recording of the word `cat`:

<figure>
    <img src="../assets/ctc_0.png" />
    <caption>Fig. 1: Valid and invalid alignments for `cat` (Source: [Distill](https://distill.pub/2017/ctc/))</caption>
</figure>

Since the final output sequence (=the transcript) is usually much shorter than the input sequence (=the frames of the spectrogram), the alignments are shortened in two steps:

1. **Deduplication**: repeated labels _between blank labels_ are collapsed into one.
2. **Blank-Removal**: the $\epsilon$-labels are removed

The following figure shows an example for an input sequence of six spectrogram frames and an output sequence of three characters (the word `cat`).

<figure>
    <img src="../assets/ctc_1.png" />
    <caption>Fig. 2: Alignment and output for example sequence (Source: [Distill](https://distill.pub/2017/ctc/))</caption>
</figure>

To allow output sequences containing two repeated labels in a row (e.g. such as two `l`s in `hello`) those must be separated by $\epsilon$ in the underlying alignment - otherwise they would be collapsed. Hence the blank token acts as a boundary between mergeable parts of the alignment. The following figure illustrates this for the example where the output word should be `hello`.

<figure>
    <img src="../assets/ctc_2.png" />
    <caption>Fig. 3: Merging of alignment parts separated by $\epsilon$ (Source: [Distill](https://distill.pub/2017/ctc/))</caption>
</figure>

Because the input length is (much) smaller than the output length, there are much more than one possible alignments which would lead to a valid output after merging. For the output sequence `cat`, the alignments `catttt`, `caatt-`, `-caatt`, `-ca-at` (and many more) would all lead to the valid output. We call the set of all alignments that would lead to a given output sequence the set of _valid alignments_.

#### Forward pass

Because the RNN will assign each input frame $x_t$ all the possible characters from the alphabet (with different probabilities), we get a vector of probabilities for each $x_t$. Since frames are processed sequentially, the vectors can be stacked to a matrix of probabilities vs. time. This matrix allows CTC to compute a score for a given output sequence from the set of valid alignments. 

The probability for a single alignment can be computed by multiplying over the probabilities for each character per time step. The probability distribution for the output sequence can then be calculated by summing up the probabilities for all valid alignments (a process also known as _marginalizing_). Consider the following example for the output sequence `cat` with an input sequence length of 6 frames. The colored lines correspond to three arbitrary alignments from the set of valid alignments. Note that since only valid alignments are shown, the nodes from the RNN that correspond to irrelevant nodes have been omitted.

<figure>
    <img src="../assets/ctc_alignments.png" />
    <caption>Fig. 4: Alignments as path through a graph (Source: [Distill](https://distill.pub/2017/ctc/))</caption>
</figure>

The probability distribution for the output sequence can then be calculated as follows:

| alignment | CTC-score |
|---|---|
| red | $0.2 \cdot 0.8 \cdot 0.1 \cdot 0.1 \cdot 0.1 \cdot 0.55 = 0.000009$ |
| green | $0.6 \cdot 0.8 \cdot 0.05 \cdot 0.4 \cdot 0.1 \cdot 0.55 = 0.000528$ |
| blue | $0.6 \cdot 0.01 \cdot 0.4 \cdot 0.1 \cdot 0.1 \cdot 0.3 = 0.000007$ |
| probability for `cat` |$\sum (score_{red} + score_{green} + score_{blue}) = 0.000544$ |

Of course for this example the probability is much too low because not all valid alignments have been considered. CTC would sum up over all valid alignments. 

#### Calculating the CTC loss by creating valid alignments with dynamic programming

One could enumerate the valid alignments by constructing all possible combinations. However, the set of valid alignments is huge. For an input sequence $X$ of length $T$ and an output sequence $Y$ of length $U$ the combinatorial rule to calculate the number of permutations is

$$\binom{T+U}{T-U}$$

For $T=100, U=50$ this number is $\approx 10^{40}$! To avoid calculating each alignment separately, a dynamic programming algorithm of merging paths is applied. To visualize this, the graph has to be adjusted by inserting additional nodes for blanks between every character in the output label. This can most easily be understood be an example:

Suppose we have the output sequence `foo` (which contains repeated labels, namely `oo`) for valid alignments over $T=6$ time steps. Since an alignment can have blanks before or after every label we can expand the label to `-f-o-o-` and construct the valid alignments by going from back to front by drawing edges in a graph. Since any alignment for `foo` can either end in `o` or a blank, we have two possible end positions to start from. Since any alignment fo `foo` can either start in `o` or a blank, we have two possible start positions to arrive at. The blanks in between can sometimes be skipped, but sometimes not. The valid alignments can be constructed by drawing edges, recursively following the following general rules for skipping characters:

1. **the previous character can be skipped**: this is the case if the previous character is an $\epsilon$ between different characters in the output label. Skipping is possible because the alignment is still valid (i.e. the output resulting from the alignment after $\epsilon$-removal would still be the same)
2. **the previous character cannot be skipped**: there are two possible reasons for this:
    - the previous character is a label in the output sequence: we cannot skip it as this would lead to an invalid alignment because a character would be missing
    - the previous character is an $\epsilon$ between repeated characters in the output label: we cannot skip it as this would lead to an invalid alignment because the repeated labels would be collapsed into one

By following the above rules we get the following graph. Each valid alignment for `foo` corresponds to a path from left to right. The two alignments with the maximum number of blanks at the beginning resp. end are highlighted.

<figure>
    <img src="../assets/ctc_alignments_dynamic.png" />
    <caption>Fig. 5: Valid alignments created with dynamic programming visualized as paths through a graph</caption>
</figure>

The scores of an alignment can be calculated recursively from the scores of the possible prefixes. Because this dynamic algorithm needs to calculate the score for each possible prefix only once, it is much more efficient than first enumerating the list of valid (complete) alignments first and then calculating the score for each alignment individually. 

The CTC-score of the whole output sequence is the sum of the two final nodes, which is just a sum of products and therefore differentiable. This means it can be used in a loss function and optimized through gradient descent.

#### Decoding
After training with above procedure we arrive at a RNN with optimized weights. This RNN can be fed new input that has never been seen before, yielding different probabilities for each character of the label alphabet at each time step. One way of inferring the output sequence would then be to create an alignment by concatenating the labels with highest probabilities in each time step and then collapse sequences of repeated labels and remove the blanks. However, this approach is greedy and would not neccessary produce the best output sequence because it is based on the probability of a single alignment. It would neglect a better sequence with valid alignments, whose individual probabilities are lower, but combined probabilities would be higher.

This problem can be solved by approximating the optimal output sequence by using beam search. For this a value $b$ for the beam width needs to be defined which controls, how many partial alignments should be followed at a time. The beam search algorithm would then start at the left and go through the time steps, only keeping the $b$ most probable partial alignments in each time step. After each time step the partial alignments can sometimes be merged by collapsing repeated characters which enables the algorithm to handle equivalent prefixes of alignments. The following figure visualizes this approach.

<figure>
    <img src="../assets/ctc_beam_search.png" />
    <caption>Fig. 6: Beam search to find an approximate optimal output sequence for a sequence of labels (Source: [Distill](https://distill.pub/2017/ctc/))</caption>
</figure>

### Appliances of CTC in selected research papers

The output produced by a well-trained model that uses CTC will usually resemble the actual audio if read out loud, but still contain a lot of spelling mistakes. Consider the following example (taken from [this video](https://www.youtube.com/watch?v=3MjIkWxXigM&t=86s)):

    Original:   TO ILLUSTRATE THE POINT A PRIMINEND MIDDLE EAST ...
    CTC-Output: TO ALSTRAIT THE POINT A PROMINENT MIDILLE EAST ..

Those mistakes would generally result in high word-error-rate (WER) and label-error-rate (LER) which can be further reduced by integrating a language model during training or to rescore the CTC scores.

Many research papers picked up the idea of CTC described in the original paper <cite data-cite="6174726/STDCGILD"></cite> and tried to refine it. Graves/Jaitlin extended the CTC as described above to reduce the WER by integrating a dictionary and a language model <cite data-cite="6174726/5Q2LHASK"></cite>. 

Using characters as labels for CTC is a rather obvious choice. However, CTC does not make assumptions about what the tokens of the output sequence should be. Sak et al. picked up this Idea and used words from a vocabulary as targets and aligned them with positions in the audio signal <cite data-cite="6174726/2STR38Y9"></cite>. However, such system often suffered from the out-of-vocabulary issue (OOV) as they can only model limited number of words. Similarly, Liu et al. tried to leverage CTC to a more general version by  removing the need for hand-crafted sets of basic units. <cite data-cite="6174726/HMQBG3C5"></cite> Instead of using a fixed-size alphabet as labels their approach was to have the network automatically learn arbitrary n-grams of basic units. Such n-grams could scale up to word-units. The idea was to get less noisy output for common words and only go down to character units (1-grams) for rare words. They consequently named their idea _Gram-CTC_.

Finally, in a rather recent paper, Li et al. proposed a hybrid approach which would solve the OOV issue by decomposing OOV words into smaller parts, which can be other freqent words or n-grams of letters. By doing so they were able to reduce the WER without the need of a language model. <cite data-cite="6174726/Z2SMUZDT"></cite>

## Summary

This chapter contains the scope and motivation for the project and also the steps taken to reach the project goal. It also explains the reasons for the chosen course of action and also the challenges that might obstruct it.

Since ASR will be a central part of the system, a selection of related research papers has been introduced. The designated implementation will be based on the CTC algorithm. Therfore its basic functionality has been explained in a bit more detail at this point.

## References
<div class="cite2c-biblio"></div>